In [1]:
# from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

from keras.layers import Embedding, Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.models import Model, Sequential

import pandas as pd
import numpy as np

import random
random.seed(1228)

from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report, confusion_matrix

%matplotlib inline

Using TensorFlow backend.


### Часть 1 [2 балла] Подготовка данных
1. Прочитайте размеченные данные Открытого корпуса, используя nltk.corpus.reader.conll.ConllCorpusReader
2. Посчитайте количество предложений и число тегов частей речи;
3. Сформируйте тестовое и обучающее множество: первые 3/4 данных – обучающее множество;

Для каждого слова:
1. Определите его окно (слова слева и справа) размера $k$;
2. Сформируйте его вектор признаков.

#### Reading the data:

In [2]:
from nltk.corpus.reader.conll import ConllCorpusReader

In [3]:
columns = ['ignore', 'words', 'ignore', 'pos', 'chunk']
train_reader = ConllCorpusReader(root = '.', fileids = 'unamb_sent_14_6.conllu', columntypes = columns)


sents = train_reader.iob_sents()
train_reader.iob_sents()

[[('«', 'PUNCT', '_'), ('Школа', 'NOUN', '_'), ('злословия', 'NOUN', '_'), ('»', 'PUNCT', '_'), ('учит', 'VERB', '_'), ('прикусить', 'VERB', '_'), ('язык', 'NOUN', '_')], [('Сохранится', 'VERB', '_'), ('ли', 'PART', '_'), ('градус', 'NOUN', '_'), ('дискуссии', 'NOUN', '_'), ('в', 'ADP', '_'), ('новом', 'ADJ', '_'), ('сезоне', 'NOUN', '_'), ('?', 'PUNCT', '_')], ...]

#### Считаем количество предложений и разных тэгов

In [4]:
len(sents) # Предложений в корпусе 38508

38508

In [4]:
#вытащим все тэги
pos = [w[1] for sent in sents for w in sent]
pos[:15]

['PUNCT',
 'NOUN',
 'NOUN',
 'PUNCT',
 'VERB',
 'VERB',
 'NOUN',
 'VERB',
 'PART',
 'NOUN',
 'NOUN',
 'ADP',
 'ADJ',
 'NOUN',
 'PUNCT']

In [6]:
# вот столько у нас разных тэгов
len(set(pos))

14

In [6]:
# а теперь посмотрим на частотное распределение тэгов 
from nltk import FreqDist
fd_pos = FreqDist(pos)

In [8]:
fd_pos

FreqDist({'ADJ': 47487,
          'ADP': 42835,
          'ADV': 13079,
          'CONJ': 21942,
          'DET': 12689,
          'INTJ': 452,
          'NOUN': 121793,
          'NUM': 10173,
          'PART': 8923,
          'PRON': 9067,
          'PROPN': 14889,
          'PUNCT': 91323,
          'VERB': 41538,
          'X': 21393})

Как мы видим, самые частые -- прилагательные и предлоги (о_О)

#### Сформируем тестовую и тренировочную выборки для тэгов

Итак, вектор POS у нас есть: `pos`. Теперь сделаем вектор признаков. Для удобства мы решили убрать информацию о границах предложений.

In [5]:
words = [word[0].lower() for sent in sents for word in sent]
words[:20]

['«',
 'школа',
 'злословия',
 '»',
 'учит',
 'прикусить',
 'язык',
 'сохранится',
 'ли',
 'градус',
 'дискуссии',
 'в',
 'новом',
 'сезоне',
 '?',
 'великолепная',
 '«',
 'школа',
 'злословия',
 '»']

Извлечение признаков:

In [7]:
embeddingsPath = './wiki.ru.vec'

In [8]:
uniq_words = set(words)
def extract_embeddings(path):
    embeddings = {}
    with open(embeddingsPath) as f:
        for line in f:
            word, vec = line.split(' ', 1)
            if word in uniq_words:
                embeddings[word] = [float(num) for num in vec.split()]
    return embeddings

embeddings = extract_embeddings(embeddingsPath)

In [9]:
embeddings['школа'][:15]

[-0.32315,
 0.91456,
 0.13797,
 0.61075,
 -0.28406,
 -0.47918,
 -0.27341,
 0.17947,
 0.54726,
 -0.47914,
 -0.20418,
 0.12833,
 0.1399,
 0.26005,
 0.53394]

In [10]:
len(embeddings['школа'])

300

In [12]:
# a strange fix
embeddings['«'] = embeddings['"']
embeddings['»'] = embeddings['"']

In [13]:
# создаём вектора для начала и конца (и плэйсхолдер на случай если в эмбеддингах слова нет)
MOF = [0] * 300

In [14]:
# k == 2
def featues_k2(tokens):
    features = []
    for i, word in enumerate(tokens):
        if word in embeddings:
            cur_word = embeddings[word]
        else:
            cur_word = MOF
        if i > 0 and tokens[i - 1] in embeddings:
            prev_word = embeddings[tokens[i - 1]]
        else:
            prev_word = MOF
        if i < len(tokens) - 1 and tokens[i + 1] in embeddings:
            next_word = embeddings[tokens[i + 1]]
        else:
            next_word = MOF
        features.append(cur_word + prev_word + next_word)
    return features

In [15]:
features = featues_k2(words)

In [16]:
len(features) == len(pos)

True

In [17]:
boundary = (len(pos) // 4) * 3
y_train = pos[:boundary]
X_train = features[:boundary]
y_test = pos[boundary:]
X_test = features[boundary:]

print(len(y_train))
print(len(y_test))

343185
114398


In [26]:
x_train = np.array(X_train)
y_train = np.array(y_train)

In [44]:
y_train.shape

(343185,)

In [58]:
x_test = np.array(X_test)

In [62]:
print(y_test[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]


### Часть 2 [4 баллов] Архитектура нейронной сети

Архитектура нейронной сети состоит из следующих слов:
1. Входной слой: нейронная сеть получает на вход вектор признаков, состоящий из $k$ конкатенированных эмбеддингов;/
2. Скрытый слой: $n_h$ нейронов и нелинейная функция активации $\theta$;
3. Выходной слой:  $|T|$ нейронов для итоговой классификации.

Обучите нейронную сеть на обучающих данных.

In [36]:
nb_filter = 250
hidden_dims = 250
nb_epoch = 10

In [55]:
le = LabelEncoder()
le.fit(list(set(pos)))
y_train = np_utils.to_categorical(le.transform(y_train), 14)
print(y_train[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


Тренируем нейросетку :3

In [56]:
model = Sequential()
model.add(Dense(128, input_shape=(900, ), activation='relu'))
model.add(Dense(14, activation = 'softmax')) # софтмакс для классификации
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=nb_epoch,  validation_split=0.1)

Train on 308866 samples, validate on 34319 samples
Epoch 1/10
308866/308866 [==============================] - 26s 86us/step - loss: 0.2172 - acc: 0.9328 - val_loss: 0.2374 - val_acc: 0.9273
Epoch 2/10
308866/308866 [==============================] - 22s 72us/step - loss: 0.1260 - acc: 0.9595 - val_loss: 0.2218 - val_acc: 0.9304
Epoch 3/10
308866/308866 [==============================] - 22s 71us/step - loss: 0.0981 - acc: 0.9677 - val_loss: 0.2385 - val_acc: 0.9303
Epoch 4/10
308866/308866 [==============================] - 24s 77us/step - loss: 0.0794 - acc: 0.9735 - val_loss: 0.2543 - val_acc: 0.9320
Epoch 5/10
308866/308866 [==============================] - 21s 69us/step - loss: 0.0671 - acc: 0.9774 - val_loss: 0.2714 - val_acc: 0.9323
Epoch 6/10
308866/308866 [==============================] - 23s 74us/step - loss: 0.0570 - acc: 0.9806 - val_loss: 0.2781 - val_acc: 0.9327
Epoch 7/10
308866/308866 [==============================] - 24s 77us/step - loss: 0.0500 - acc: 0.9830 - val_

Ура, обучили!

### Часть 3 [1 балл] Оценка качества

Протестируйте нейронную сеть на тестовых данных. Используйте accuracy для оценки качества модели.

In [59]:
y_pred = model.predict_classes(x_test)
y_pred = le.inverse_transform(y_pred)

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [65]:
y_test = pos[boundary:]
print(y_pred[0])

VERB


In [70]:
print(classification_report(y_test, y_pred))
accuracy_score(y_test, y_pred)

             precision    recall  f1-score   support

        ADJ       0.96      0.93      0.94     12573
        ADP       1.00      0.99      0.99     10039
        ADV       0.92      0.89      0.91      3042
       CONJ       0.97      0.98      0.97      5510
        DET       0.95      0.97      0.96      2794
       INTJ       0.65      0.60      0.62       106
       NOUN       0.96      0.98      0.97     32653
        NUM       0.70      0.79      0.74      2283
       PART       0.94      0.92      0.93      1906
       PRON       0.98      0.98      0.98      1896
      PROPN       0.86      0.78      0.81      2734
      PUNCT       0.98      0.97      0.97     23948
       VERB       0.97      0.97      0.97      9458
          X       0.68      0.67      0.67      5456

avg / total       0.94      0.94      0.94    114398



0.9442735012849875

Похоже, получилось довольно неплохо!

### Часть 4 [1 балл] Оптимизация гиперпарметров

В эксперименте участвуют следующие гиперпараметры:
* $k$ – размер окна;
* $n_h$ – число нейронов на скрытом слое;
* $\theta$ – вид функции активации.

Оцените их влияние на качество модели. Как увеличение окна или числа нейронов влияет на итоговый показатель качества? Зависит ли итоговый показатель качества от функции активации на скрытом слое? 

In [71]:
# фц которая перебирает разные параметры
def train_model(n, theta):
    model = Sequential()
    model.add(Dense(n, input_shape=(3*300,), activation = theta)) # скрытый слой; первый аргумент - размерность аутпута
    model.add(Dense(14, activation = 'softmax')) 
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=nb_epoch, batch_size=batch_size,  validation_split=0.1)

### Часть 5 [2 балла] Анализ ошибок
1. Приведите примеры из тестового множества, на которых нейронная сеть ошибается. Объясните, почему возникают ошибки.
2. Протестируйте нейронную сеть на произвольном предложении (не из тестовых данных). Возникают ли ошибки? Почему?

In [76]:
# 20 токенов, на которых классификатор ошибается
count = 0
print('word', '\t\t', 'true', '\t\t', 'predicted')
for i in range(len(y_test)):
    if y_test[i] != y_pred[i]:
        count += 1
        print(words[boundary+i], '\t\t', y_test[i], '\t\t', y_pred[i])
    if count == 20:
        break

word 		 true 		 predicted
какой-то 		 DET 		 X
травлю 		 VERB 		 NOUN
адидас 		 X 		 PROPN
что 		 CONJ 		 PART
: 		 PUNCT 		 X
1999 		 NUM 		 X
теде 		 PROPN 		 X
права 		 ADJ 		 NOUN
насчет 		 ADP 		 VERB
%) 		 PUNCT 		 X
заплутал 		 VERB 		 NUM
петера 		 PROPN 		 X
психотест 		 X 		 NOUN
как 		 CONJ 		 ADV
надо 		 ADV 		 VERB
программно 		 ADV 		 NOUN
общем 		 ADJ 		 X
кладу 		 VERB 		 NOUN
рядом 		 ADV 		 ADP
:) 		 PUNCT 		 NUM


Как мы видим, у классификатора проблемы со смайликами (неудивительно, всё-таки, это на просто пунктуация).

In [81]:
custom_sent = 'дорогая , я что-то нажала и всё исчезло !'.split()
fiches = featues_k2(custom_sent)
pred = model.predict_classes(np.array(fiches))
le.inverse_transform(pred)

/usr/local/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array(['ADJ', 'PUNCT', 'PRON', 'X', 'NUM', 'CONJ', 'NOUN', 'VERB',
       'PUNCT'], dtype='<U5')

Не впечатляет... Но большая часть токенов предсказана правильно.